In [1]:
import requests
import time
import re
import openpyxl
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [2]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/prodazha-zdaniy?per-page=50'

Добыча колличества ссылок

In [3]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 50))

print(count)

75


Составляем массив ссылок

In [4]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}&page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 3/3 [00:01<00:00,  2.85it/s]

75


Составление своей базы данных

In [5]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')
 
    try:
        w2 = soup.find('table', id = 'w2')
        if w2:
            w2 = w2.find_all('tr')
            for tr in w2:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('w2 not found')

    try:
        properties = soup.find('div', class_ = 'properties m-t-2')
        if properties:
            properties = properties.find_all('div', class_ = 'property')
            for property in properties:
                key = property.find('div', class_ = 'property-name').text.strip()
                val = property.find('div', class_ = 'property-value').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('properties m-t-2 not found')
    
    try:
        buildings = soup.find('div', 'around').find_all('div', 'group')
        for building in buildings:
            name = building.find('div','group-body').find('span').text
            dist = building.find('div','group-body').find('div', 'company').find('span', 'company__distance').text
            tmp[name] = dist   
    except:
        pass

    df = pd.concat([df, pd.DataFrame(tmp)], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Комерческая недвижимость'
df['Подсегмент'] = 'Покупка зданий'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 75/75 [00:18<00:00,  4.00it/s]


Посмотрим на результаты вкрадце

In [6]:
df.head()

,Ссылка,Заголовок,Цена,Описание,№,Район,Адрес,Площадь,Этажность,Материал стен,Объект,Назначение,Торг уместен,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/prodazha-zdaniy/prosp-g...,"Продажа здания на просп. Гагарина, д. 168Приок...",160 000 000 руб.,Продается отдельностоящее здание общей площадь...,2707348,Приокский,"просп. Гагарина, д. 168",6 300 м²,5,Уточнить,отдельно стоящее здание,офисное,NaN,gipernn,Комерческая недвижимость,Покупка зданий,Нижний Новгород,2022-12-22
1,https://www.gipernn.ru/prodazha-zdaniy/ul-delo...,"Продажа здания на ул. Деловая, д. 15Нижегородс...",30 600 000 руб.,"Продается здание бизнес-центра 2-х этажное, пл...",2838726,Нижегородский,"ул. Деловая, д. 15",657 м²,2,Уточнить,бизнес-центр,офисное,NaN,gipernn,Комерческая недвижимость,Покупка зданий,Нижний Новгород,2022-12-22
2,https://www.gipernn.ru/prodazha-zdaniy/bolshay...,Продажа здания на Большая Починковcкая улицаСо...,21 000 000 руб.,"Действующее кафе Регина"", заведение работает и...",2835986,Сормовский,Большая Починковcкая улица,655 м²,2,Уточнить,отдельно стоящее здание,общепит,NaN,gipernn,Комерческая недвижимость,Покупка зданий,Нижний Новгород,2022-12-22
3,https://www.gipernn.ru/prodazha-zdaniy/ul-mech...,"Продажа здания на ул. Мечникова, д. 65аМосковс...",20 000 000 руб.,Продажа отдельно стоящего двухэтажного здания ...,2845579,Московский,"ул. Мечникова, д. 65а",680 м²,1,кирпич,отдельно стоящее здание,"общепит, производственное",NaN,gipernn,Комерческая недвижимость,Покупка зданий,Нижний Новгород,2022-12-22
4,https://www.gipernn.ru/prodazha-zdaniy/ul-svet...,Продажа здания на ул. СветлоярскаяСормовский р...,29 000 000 руб.,Продажа одноэтажного здания в Сормовском район...,2843418,Сормовский,ул. Светлоярская,304 м²,1,комбинированный,отдельно стоящее здание,"торговое, автобизнес",NaN,gipernn,Комерческая недвижимость,Покупка зданий,Нижний Новгород,2022-12-22


Экспортируем таблицу

In [7]:
df.to_excel('buildings-buy.xlsx', index=False)